# Day 9: Smoke Basin

## Part 1

These caves seem to be [lava tubes](https://en.wikipedia.org/wiki/Lava_tube). Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

<pre>
<code>
2199943210
3987894921
9856789892
8767896789
9899965678
</code>
</pre>

Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the **low points** - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are **four** low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The **risk level** of a low point is **1 plus its height**. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore **`15`**.

Find all of the low points on your heightmap. **What is the sum of the risk levels of all low points on your heightmap?**

In [9]:
data = []
with open('day09-input01.txt') as f:
    data = f.readlines()
    data = [[int(char) for char in line.strip()] for line in data]

def is_lowest_point(i, j, data):
    isLowest = True;
    current_val = data[i][j]
    
    if i > 0:
        if data[i-1][j] <= current_val:
            isLowest = False
    if j > 0:
        if data[i][j-1] <= current_val:
            isLowest = False
    if i < len(data)-1:
        if data[i+1][j] <= current_val:
            isLowest = False
    if j < len(data[0])-1:
        if data[i][j+1] <= current_val:
            isLowest = False

    return isLowest

low_points_risk = []

for i in range(len(data)):
    for j in range(len(data[0])):
        if is_lowest_point(i, j, data):
            low_points_risk.append(data[i][j] + 1)

print(sum(low_points_risk))


631


## Part 2

Next, you need to find the largest basins so you know what areas are most important to avoid.

A **basin** is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height `9` do not count as being in any basin, and all other locations will always be part of exactly one basin.

The **size** of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

<pre>
<code>
2199943210
3987894921
9856789892
8767896789
9899965678
</code>
</pre>

The top-right basin, size `9`:

<pre>
<code>
2199943210
3987894921
9856789892
8767896789
9899965678
</code>
</pre>

The middle basin, size `14`:

<pre>
<code>
2199943210
3987894921
9856789892
8767896789
9899965678
</code>
</pre>

The bottom-right basin, size `9`:

<pre>
<code>
2199943210
3987894921
9856789892
8767896789
9899965678
</code>
</pre>

Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = 1134`.

**What do you get if you multiply together the sizes of the three largest basins?**

In [28]:
data = []
with open('day09-input01.txt') as f:
    data = f.readlines()
    data = [[int(char) for char in line.strip()] for line in data]
    
basins = []

def get_neighbors(data, i, j):
    return { x:data[x[0]][x[1]] for x in [(i, j-1), (i, j+1), (i-1, j), (i+1, j)] if -1 not in x and 100 not in x and data[x[0]][x[1]] != 9 }

def check_neighbors(data, current_basin_points, more_neighbors):
    all_neighbors = {}

    for i, j in more_neighbors.keys():
        new_neighbors = { x:data[x[0]][x[1]] for x in [(i, j-1), (i, j+1), (i-1, j), (i+1, j)] if -1 not in x and 100 not in x and data[x[0]][x[1]] != 9 and x not in current_basin_points.keys() }
        all_neighbors = { **all_neighbors, **new_neighbors }

    return all_neighbors

def get_basins_by_size(data):
    basin_list = []
    for i in range(len(data)):
        for j in range(len(data[0])):
            starting_value = data[i][j]

            if (i,j) not in [basin_list[i][j][0] for i in range(len(basin_list)) for j in range(len(basin_list[i]))] and starting_value != 9:
                neighbors = get_neighbors(data, i, j)
                current_basin_points = { **{ (i,j): starting_value}, **neighbors }
                more_neighbors = check_neighbors(data, current_basin_points, neighbors)
                current_basin_points = { **current_basin_points, **more_neighbors }

                while more_neighbors != {}:
                    more_neighbors = check_neighbors(data, current_basin_points, more_neighbors)
                    current_basin_points = { **current_basin_points, **more_neighbors }

                basin_list.append(sorted(current_basin_points.items()))
            else:
                continue

    out = [len(bl) for bl in basin_list]
    out.sort()
    return out

basins = get_basins_by_size(data)

top_three = basins[-3:]
print(top_three[0]*top_three[1]*top_three[2])


821560
